In [ ]:
# 汇总各项能量构建 KS 哈密顿矩阵
%run basis_func_with_kinetic.ipynb
%run Hartree.ipynb
%run Vex.ipynb
%run Vps_lc.ipynb
%run Vps_nl.ipynb

import numpy as np
from typing import Dict, List, Sequence, Tuple


def _max_plane_wave_index(coeffs: Sequence[Tuple[int, int, int]]) -> int:
    """返回平面波系数 (n1,n2,n3) 的绝对值最大整数索引，用于确定所需 FFT 网格大小。"""
    if not coeffs:
        return 0
    arr = np.abs(np.asarray(coeffs, dtype=int))
    return int(arr.max())


def _next_power_of_two(n: int) -> int:
    """给定整数 n，返回大于等于 n 的 2 的幂次，用于保证 FFT 高效执行。"""
    if n <= 1:
        return 2
    return 1 << (int(np.ceil(np.log2(n))))


def _fft_index_lookup(Nk: int) -> Dict[int, int]:
    """构造整数傅里叶指数到 numpy FFT 网格索引的映射，便于查找 V(G)。"""
    freq = np.fft.fftfreq(Nk) * Nk
    freq_int = np.round(freq).astype(int)
    return {int(val): idx for idx, val in enumerate(freq_int)}


def reciprocal_potential_matrix(potential_G: np.ndarray,
                                coeffs: Sequence[Tuple[int, int, int]]) -> np.ndarray:
    """根据倒空间势能 V(G) 构造平面波矩阵元素 V(G_i - G_j)。"""
    if potential_G.ndim != 3 or potential_G.shape[0] != potential_G.shape[1] or potential_G.shape[1] != potential_G.shape[2]:
        raise ValueError('potential_G must be a cubic grid with shape (Nk, Nk, Nk).')
    Nk = potential_G.shape[0]
    idx_map = _fft_index_lookup(Nk)
    n_pw = len(coeffs)
    mat = np.zeros((n_pw, n_pw), dtype=complex)
    for i, ci in enumerate(coeffs):
        for j, cj in enumerate(coeffs):
            delta = (ci[0] - cj[0], ci[1] - cj[1], ci[2] - cj[2])
            try:
                mat[i, j] = potential_G[
                    idx_map[delta[0]],
                    idx_map[delta[1]],
                    idx_map[delta[2]],
                ]
            except KeyError as exc:
                raise ValueError(
                    f"Reciprocal grid (Nk={Nk}) too small for plane-wave difference {delta}"
                ) from exc
    return mat


def determine_fft_grid_size(coeffs: Sequence[Tuple[int, int, int]], base: int = 32) -> int:
    """根据平面波最高指数自动给出 FFT 网格大小（偶数的 2 次幂）。"""
    nmax = _max_plane_wave_index(coeffs)
    target = max(base, 2 * nmax + 1)
    Nk = _next_power_of_two(target)
    if Nk % 2 == 1:
        Nk += 1
    return Nk


def local_pseudopotential_matrix(coeffs: Sequence[Tuple[int, int, int]],
                                 b_vecs: np.ndarray,
                                 Z: float = 3.0,
                                 r_c: float = 0.6,
                                 r_gau: float = 1.0,
                                 Nr: int = 2000) -> np.ndarray:
    """计算局域赝势在平面波基的矩阵元，使用短程数值积分+长程解析补偿。

    参数:
        coeffs: 平面波整数系数列表。
        b_vecs: 倒格矢矩阵。
        Z, r_c, r_gau, Nr: 控制示例赝势形状与积分精度的参数。
    返回:
        n_pw×n_pw 的实对称矩阵，对应 V_ps,loc。无平面波时返回空矩阵。
    """
    if not coeffs:
        return np.zeros((0, 0), dtype=float)
    delta_norm_map: Dict[Tuple[int, int, int], float] = {}
    b1, b2, b3 = b_vecs
    for ci in coeffs:
        for cj in coeffs:
            delta = (ci[0] - cj[0], ci[1] - cj[1], ci[2] - cj[2])
            if delta in delta_norm_map:
                continue
            vec = delta[0] * b1 + delta[1] * b2 + delta[2] * b3
            delta_norm_map[delta] = np.linalg.norm(vec)
    deltas = list(delta_norm_map.keys())
    norms = np.array([delta_norm_map[d] for d in deltas])
    V_short = Vps_loc_short_G_of_Gnorms(norms, Z=Z, r_c=r_c, r_gau=r_gau, Nr=Nr)
    V_gauss = Vgauss_G_analytic(norms, Z=Z, r_gau=r_gau)
    V_total = V_short - V_gauss
    V_total[np.where(norms < 1e-12)] = 0.0
    delta_to_value = {delta: val for delta, val in zip(deltas, V_total)}
    n_pw = len(coeffs)
    mat = np.zeros((n_pw, n_pw), dtype=float)
    for i, ci in enumerate(coeffs):
        for j, cj in enumerate(coeffs):
            delta = (ci[0] - cj[0], ci[1] - cj[1], ci[2] - cj[2])
            mat[i, j] = delta_to_value[delta]
    return mat


def assemble_ks_hamiltonian(a0: float = 4.05,
                            Ecut_eV: float = 200.0,
                            kvec: np.ndarray | None = None,
                            sigma_bohr: float = 1.0,
                            Nk_override: int | None = None,
                            Z_loc: float = 3.0,
                            r_c: float = 0.6,
                            r_gau: float = 1.0,
                            projectors: List[GaussianProjectorChannel] | None = None) -> Dict[str, np.ndarray]:
    """汇集动能、Hartree、交换、局域/非局域赝势，生成给定 k 点的 KS 哈密顿矩阵。

    参数:
        a0, Ecut_eV, kvec: 晶格常数、动能截断、k 点设定。
        sigma_bohr, Nk_override: 控制密度构造及 FFT 网格。
        Z_loc, r_c, r_gau: 局域赝势模型参数。
        projectors: 非局域赝势投影子列表，默认构造一个示例。
    返回:
        包含晶格向量、平面波列表、各势能分量矩阵以及总哈密顿矩阵的字典。
    """
    if kvec is None:
        kvec = np.zeros(3)
    a_vecs = primitive_fcc(a0)
    b_vecs, Omega = reciprocal_vectors(a_vecs)
    Gs, coeffs, kinetic = generate_G_vectors(b_vecs, Ecut_eV, kvec=kvec)
    if Gs.size == 0:
        raise RuntimeError('No plane waves generated. Increase Ecut or check lattice.')
    Nk = Nk_override or determine_fft_grid_size(coeffs)
    rho_grid, Omega_density, _, _, _ = build_gaussian_density(a0=a0, Nk=Nk, sigma_bohr=sigma_bohr)
    nG, Ggrid = density_grid_to_reciprocal(rho_grid, b_vecs)
    Gnorm2 = np.sum(Ggrid**2, axis=-1)
    vH_G = np.zeros_like(nG, dtype=complex)
    mask = Gnorm2 > 1e-10
    vH_G[mask] = 4.0 * np.pi * nG[mask] / Gnorm2[mask]
    vH_G[~mask] = 0.0
    _, v_x = lda_exchange_potential(rho_grid)
    vx_G, _ = exchange_potential_to_reciprocal(v_x, b_vecs)
    vH_matrix = reciprocal_potential_matrix(vH_G, coeffs).real
    vx_matrix = reciprocal_potential_matrix(vx_G, coeffs).real
    Vloc_matrix = local_pseudopotential_matrix(coeffs, b_vecs, Z=Z_loc, r_c=r_c, r_gau=r_gau)
    if projectors is None:
        projectors = [
            GaussianProjectorChannel(l=0, D_l=-2.5, r_cut=1.1),
            GaussianProjectorChannel(l=1, D_l=-1.2, r_cut=1.3),
        ]
    _, Vnl_matrix = nonlocal_pseudopotential_matrix(
        a_vecs=a_vecs,
        atom_positions=[np.zeros(3)],
        projectors=projectors,
        Ecut_eV=Ecut_eV,
        kvec=kvec,
        Gvecs_override=Gs,
    )
    kinetic_matrix = np.diag(kinetic)
    H_total = kinetic_matrix + vH_matrix + vx_matrix + Vloc_matrix + Vnl_matrix
    return {
        'a_vecs': a_vecs,
        'b_vecs': b_vecs,
        'G_vectors': Gs,
        'coeffs': coeffs,
        'kinetic': kinetic,
        'components': {
            'T': kinetic_matrix,
            'V_H': vH_matrix,
            'V_x': vx_matrix,
            'V_ps_loc': Vloc_matrix,
            'V_ps_nl': Vnl_matrix,
        },
        'H_total': H_total,
        'Nk_grid': Nk,
    }


if __name__ == '__main__':
    ks_data = assemble_ks_hamiltonian(a0=4.05, Ecut_eV=200.0)
    H_mat = ks_data['H_total']
    H_herm = 0.5 * (H_mat + H_mat.conj().T)
    eigvals = np.linalg.eigvalsh(H_herm)
    print(f"Plane waves: {H_mat.shape[0]}")
    print(f"FFT grid Nk: {ks_data['Nk_grid']}")
    print('First ten eigenvalues (Hartree):')
    print(eigvals[:10])


Plane waves: 113
FFT grid Nk: 32
First ten eigenvalues (Hartree):
[-89.5376748  -83.36115521 -81.32319435 -70.00774496 -69.92352091
 -69.92352091 -69.77074564 -69.77074564 -69.75414952 -57.77096946]
[[-5.84482666e-01+0.j -2.77471023e+01+0.j -2.77471023e+01+0.j ...
  -3.54587855e-01+0.j -3.54587855e-01+0.j -3.54587855e-01+0.j]
 [-2.77471023e+01+0.j  5.27202113e-01+0.j  4.82696029e+00+0.j ...
  -1.06145050e+00+0.j -4.28573627e-01+0.j -3.47448265e+00+0.j]
 [-2.77471023e+01+0.j  4.82696029e+00+0.j  5.27202113e-01+0.j ...
  -3.47448265e+00+0.j -1.97932923e-01+0.j -1.06145050e+00+0.j]
 ...
 [-3.54587855e-01+0.j -1.06145050e+00+0.j -3.47448265e+00+0.j ...
   6.62277805e+00+0.j -5.49624577e-03+0.j  4.88791344e+00+0.j]
 [-3.54587855e-01+0.j -4.28573627e-01+0.j -1.97932923e-01+0.j ...
  -5.49624577e-03+0.j  6.62277805e+00+0.j -9.88404282e-03+0.j]
 [-3.54587855e-01+0.j -3.47448265e+00+0.j -1.06145050e+00+0.j ...
   4.88791344e+00+0.j -9.88404282e-03+0.j  6.62277805e+00+0.j]]
